<a href="https://colab.research.google.com/github/alessandronicolini/Incremental-Learning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import important files from github 


In [ ]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/Incremental-Learning.git # clone proj repository
!rm -rf Incremental-Learning/README.md 
!rm -rf Incremental-Learning/baselines.ipynb
sys.path.append('Incremental-Learning/')

Cloning into 'Incremental-Learning'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 41 (delta 16), reused 25 (delta 9), pack-reused 0
Unpacking objects: 100% (41/41), done.


import libraries


In [13]:
import torch
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import numpy as np

# project classes --------------------------------------------------------------
from ilcifar100 import ilCIFAR100
from resnet_cifar import resnet32

Define params

In [10]:
NUM_RUN = 3
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 128
NUM_CLASS_BATCH = 10

Define train transform and test transform

In [14]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders 

In [15]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(NUM_CLASS_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    original_training_set = ilCIFAR100(root="dataset", seed=SEEDS[run], transform = train_transform)
    original_test_set = ilCIFAR100(root="dataset", seed=SEEDS[run], transform=test_transform)
    
    for class_batch in range(NUM_CLASS_BATCH):
        
        # get indices
        train_indices = original_training_set.batches[class_batch]['train']
        val_indices = original_training_set.batches[class_batch]['val']
        test_indices = original_test_set.batches[class_batch]

        # make subsets
        train_set = Subset(dataset=original_training, indices=train_indices)
        val_set = Subset(dataset=original_training, indices=val_indices)
        test_set = Subset(dataset=original_test, indices=test_indices)

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        train_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

        # update dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


FINE TUNING 

In [ ]:
!rm -r Incremental-Learning/